In [95]:
from warnings import filterwarnings as fw; fw('ignore')
from sklearn.inspection import permutation_importance
from pycaret.classification import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import shap

In [96]:
df = pd.read_csv('dataset_t0.5.csv', index_col='CandidateUniqueId')
df.drop(columns=['Performance', 'Job Status'], inplace=True)
# df['Overall'] = df['Overall'].replace({'Low_Attrite': 0, 'Low_Active': 1, 'High_Attrite': 2, 'High_Active': 3})
X = df.drop('Overall', axis=1)
y = df['Overall']

print(len(df))
df.head()

937


,25095_87335,25095_87336,25095_87337,25096_87338,25096_87339,25096_87340,25097_87341,25097_87342,25098_87343,25098_87344,...,25135_87451,25136_87452,25136_87453,25137_87454,25137_87455,25138_87456,25138_87457,25139_87458,25139_87459,Overall
CandidateUniqueId,,,,,,,,,,,,,,,,,,,,,
1038047,1,0,0,1,0,0,0,1,0,1,...,0,1,0,0,1,0,1,0,1,Low_Active
1039489,1,0,0,1,0,0,0,1,0,1,...,0,1,0,1,0,1,0,0,1,Low_Active
1039490,1,0,0,1,0,0,0,1,0,1,...,0,1,0,1,0,1,0,0,1,Low_Active
1039493,0,0,1,1,0,0,0,1,0,1,...,0,1,0,0,1,1,0,0,1,Low_Active
1039506,1,0,0,1,0,0,0,1,0,1,...,0,1,0,1,0,0,1,0,1,Low_Active


In [97]:
set_config('seed', 69)
s = setup(df, target='Overall', numeric_features=df.columns[:-1].tolist(), fold=5,
        train_size=1, data_split_shuffle=True, data_split_stratify=True, fold_shuffle=True, 
        index=False, preprocess=False, test_data=df, experiment_name='pmaps')

,Description,Value
0,Session id,8614
1,Target,Overall
2,Target type,Multiclass
3,Original data shape,"(1874, 102)"
4,Transformed data shape,"(1874, 102)"
5,Transformed train set shape,"(937, 102)"
6,Transformed test set shape,"(937, 102)"
7,Numeric features,101


In [90]:
# best_models = compare_models(sort='F1', n_select=1)
model = create_model('knn')
tuned_model = tune_model(model, optimize='F1', round=4, n_iter=100)
# evaluate_model(tuned_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5053,0.5534,0.5053,0.4785,0.4805,0.0263,0.0268
1,0.5319,0.5295,0.5319,0.4828,0.5008,0.0690,0.0710
2,0.5027,0.5254,0.5027,0.4500,0.4744,0.0142,0.0145
3,0.5294,0.5442,0.5294,0.4759,0.5012,0.0798,0.0813
4,0.5561,0.5166,0.5561,0.4829,0.5130,0.0860,0.0907
Mean,0.5251,0.5338,0.5251,0.4740,0.4940,0.0551,0.0569
Std,0.0196,0.0132,0.0196,0.0123,0.0143,0.0292,0.0305


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6064,0.5795,0.6064,0.5811,0.5731,0.1968,0.2075
1,0.5372,0.5640,0.5372,0.4839,0.5023,0.0708,0.0738
2,0.4920,0.5361,0.4920,0.4321,0.4573,-0.0185,-0.0191
3,0.5401,0.5506,0.5401,0.4794,0.5070,0.0809,0.0831
4,0.5615,0.4911,0.5615,0.4778,0.5051,0.0702,0.0778
Mean,0.5474,0.5443,0.5474,0.4908,0.5090,0.0800,0.0846
Std,0.0372,0.0302,0.0372,0.0489,0.0370,0.0686,0.0722


Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [91]:
final_model = finalize_model(tuned_model)
model = final_model.named_steps['actual_estimator']
# pickle.dump(model, open('max_data_knn_model_sklearn.pkl', 'wb'))
# save_model(final_model, 'max_data_knn_model_pycaret')

In [80]:
# plot_model(final_model, plot = 'feature_all', save=True)
plot_model(final_model, plot = 'confusion_matrix', save=True)
plot_model(final_model, plot = 'class_report', save=True)

weights = np.abs(permutation_importance(model, X, y, n_repeats=10, random_state=69).importances_mean)
features = model.feature_names_in_
feature_weights = zip(features, weights/sum(weights))
feature_weights = sorted(feature_weights, key=lambda x: x[1], reverse=True)
fw = pd.DataFrame(feature_weights, columns=['Feature', 'Importance'])
fw.to_csv('feature_importance.csv', index=False)

# sorted_features, sorted_importances = zip(*feature_weights)
# plt.figure(figsize=(10, 50))
# plt.barh(range(len(sorted_importances)), sorted_importances, align='center')
# plt.yticks(range(len(sorted_features)), sorted_features)
# plt.xlabel('Importance')
# plt.ylabel('Features')
# plt.title('Feature Importance')
# plt.gca().invert_yaxis()
# plt.savefig('Feature Importance.png')
# plt.close()

print("Plotted")

Plotted


In [92]:
try: predict_model(final_model)
except: pass

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,K Neighbors Classifier,0.8933,0.9777,0.8933,0.8985,0.8922,0.8053,0.8072


In [98]:
preds = actual_model.predict(X)
probs = actual_model.predict_proba(X)
probs = list(map(lambda x: round(max(x),2), probs))
# prob_str = list(map(lambda x: ', '.join(map(lambda y: str(round(y,2)), x)), probs))
score = actual_model.score(X, y)

df['Predicted'] = preds
df['Probabilities'] = probs
print(f'Model Accuracy: {score}')
df.head()

Model Accuracy: 0.6456776947705443


,25095_87335,25095_87336,25095_87337,25096_87338,25096_87339,25096_87340,25097_87341,25097_87342,25098_87343,25098_87344,...,25136_87453,25137_87454,25137_87455,25138_87456,25138_87457,25139_87458,25139_87459,Overall,Predicted,Probabilities
CandidateUniqueId,,,,,,,,,,,,,,,,,,,,,
1038047,1,0,0,1,0,0,0,1,0,1,...,0,0,1,0,1,0,1,Low_Active,Low_Active,0.73
1039489,1,0,0,1,0,0,0,1,0,1,...,0,1,0,1,0,0,1,Low_Active,Low_Active,0.64
1039490,1,0,0,1,0,0,0,1,0,1,...,0,1,0,1,0,0,1,Low_Active,High_Active,0.55
1039493,0,0,1,1,0,0,0,1,0,1,...,0,0,1,1,0,0,1,Low_Active,Low_Active,0.73
1039506,1,0,0,1,0,0,0,1,0,1,...,0,1,0,0,1,0,1,Low_Active,High_Active,0.45


In [99]:
df['Performance'] = list(map(lambda x: x.split('_')[0] , df['Overall']))
df['Attrition'] = list(map(lambda x: x.split('_')[1] , df['Overall']))
df['Joint'] = df['Overall']

df['Performance_Predicted'] = list(map(lambda x: x.split('_')[0] , df['Predicted']))
df['Attrition_Predicted'] = list(map(lambda x: x.split('_')[1] , df['Predicted']))
df['Joint_Predicted'] = df['Predicted']

df['Probability_Score'] = df['Probabilities']
df.drop(columns=['Overall', 'Predicted', 'Probabilities'], inplace=True)
df = df[['Performance', 'Attrition', 'Performance_Predicted', 'Attrition_Predicted', 'Probability_Score']]
df.to_csv('predictions.csv')
print(len(df))
df.head()

937


,Performance,Attrition,Performance_Predicted,Attrition_Predicted,Probability_Score
CandidateUniqueId,,,,,
1038047,Low,Active,Low,Active,0.73
1039489,Low,Active,Low,Active,0.64
1039490,Low,Active,High,Active,0.55
1039493,Low,Active,Low,Active,0.73
1039506,Low,Active,High,Active,0.45
